## Example Usage of the GMail Python API

This Notebook shows the basic usage of the GMail API

1. Load E-Mails from the server

2. Send E-Mails with the server

In order to be able to use the API you need a valid token as a .pickle file. In the first initialization of the GMail service you need to pass the path to this token.pickle file. Change the parameter below to your file to rerun the examples. 


For furhter information on how to send attachments, ... please consult the Documentation: 

https://googleapis.github.io/google-api-python-client/docs/epy/index.html

and the Guide: 

https://developers.google.com/gmail/api/quickstart/python

In [1]:
# !pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
# !pip install typing

In [2]:
from googleapiclient.discovery import build, Resource
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import base64
import pickle
import os
from typing import Dict
import email
import pandas as pd
import numpy as np
import torch
from transformers import *
from scipy.special import softmax

### Basic Functions

In [3]:
def load_service(token_path:str):
    """ Load the service from the specifies token file

    Args:
        token_path (str): Authorized Gmail API service instance.
    """
    if os.path.exists(token_path):
        with open(token_path, 'rb') as token:
            creds = pickle.load(token)
            service = build('gmail', 'v1', credentials=creds)
            return service

    else: 
        print("Credentials not found")
        return None

In [4]:
def send_message(service:Resource, user_id:str, message:Dict[str,str]):
    """Send an email message.

    Args:
        service (Resource): Authorized Gmail API service instance.
        user_id (str): User's email address. The special value "me" can be used to indicate the authenticated user.
        message (Dict[str,str]): Message to be sent.
    """
    try:
        message = (service.users().messages().send(userId=user_id, body=message)
                  .execute())
        print('Message Id: %s' % message['id'])
    except Exception as e:
        print('An error occurred: %s' % e)

# Create Message

In [5]:
def create_message(sender:str, to:str, subject:str, message_text:str) -> Dict[str,str]:
  """Create a message for an email.

  Args:
    sender (str): Email address of the sender.
    to (str): Email address of the receiver.
    subject (str): The subject of the email message.
    message_text (str): The text of the email message.

  Returns:
    Dict[str,str]: An object containing a base64url encoded email object.
  """
  message = MIMEMultipart() #MIMEText(message_text, "html", _charset="utf-8")
  #message.set_payload(message_text, charset="utf-8")
  message['to'] = to
  message['from'] = sender
  message['subject'] = subject
  message.attach(MIMEText(message_text, _charset='utf-8'))

  print(message)

  encoded_message = base64.urlsafe_b64encode(message.as_bytes())
  print(encoded_message)
  print(encoded_message.decode())
  return {'raw': encoded_message.decode()}

In [6]:
message = create_message("datathon@lumos-consulting.at", "team4@lumos-consulting.at", "Test1","Tasersterö")
# message = create_message("team4@lumos-consulting.at", "team4@lumos-consulting.at", "Test_1", "Ich habe WAREHOUSE Produkte im IT&OFFICEKatalog15 firstclass standardmaessig installiert. Die Installation lief fehlerfrei ab und wurde ordnungsgemaess abgeschlossen. Das Programm wurde anschliessend registriert.  Beim Programmaufruf kommt fuer ca. 4-5 Sekunden die Sanduhr auf dem Bildschirm, sonst keine weitere Reaktionen, keine Fehlermeldungen, Warnungen etc. Das Programm wird nicht gestartet.")
# message

Content-Type: multipart/mixed; boundary="===============6511784582562386666=="
MIME-Version: 1.0
to: team4@lumos-consulting.at
from: datathon@lumos-consulting.at
subject: Test1

--===============6511784582562386666==
Content-Type: text/plain; charset="utf-8"
MIME-Version: 1.0
Content-Transfer-Encoding: base64

VGFzZXJzdGVyw7Y=

--===============6511784582562386666==--

b'Q29udGVudC1UeXBlOiBtdWx0aXBhcnQvbWl4ZWQ7IGJvdW5kYXJ5PSI9PT09PT09PT09PT09PT02NTExNzg0NTgyNTYyMzg2NjY2PT0iCk1JTUUtVmVyc2lvbjogMS4wCnRvOiB0ZWFtNEBsdW1vcy1jb25zdWx0aW5nLmF0CmZyb206IGRhdGF0aG9uQGx1bW9zLWNvbnN1bHRpbmcuYXQKc3ViamVjdDogVGVzdDEKCi0tPT09PT09PT09PT09PT09NjUxMTc4NDU4MjU2MjM4NjY2Nj09CkNvbnRlbnQtVHlwZTogdGV4dC9wbGFpbjsgY2hhcnNldD0idXRmLTgiCk1JTUUtVmVyc2lvbjogMS4wCkNvbnRlbnQtVHJhbnNmZXItRW5jb2Rpbmc6IGJhc2U2NAoKVkdGelpYSnpkR1Z5dzdZPQoKLS09PT09PT09PT09PT09PT02NTExNzg0NTgyNTYyMzg2NjY2PT0tLQo='
Q29udGVudC1UeXBlOiBtdWx0aXBhcnQvbWl4ZWQ7IGJvdW5kYXJ5PSI9PT09PT09PT09PT09PT02NTExNzg0NTgyNTYyMzg2NjY2PT0iCk1JTUUtVmVyc2lvbjogMS4w

In [7]:
def receive_unread_messages(service:Resource):
    """ Recieve all unread messages and print them

        Args:
        service (Resource): Authorized Gmail API service instance.

    """
    messages = []
    # request a list of all the messages 
    result = service.users().messages().list(userId='me').execute() 
  
    # iterate through all the messages 
    for msg in result.get('messages') : 
        # Get the message from its id   
        # Use try-except to avoid any Errors 
        message = service.users().messages().get(userId="me", id=msg["id"],
                                             format='raw').execute()
        
        
        if "UNREAD" in message["labelIds"]:
            try:
                print(message["id"])
                msg_str = base64.urlsafe_b64decode(message['raw'].encode("utf-8")).decode("utf-8")
                print(msg_str)
                mime_msg = email.message_from_string(msg_str)
                
                messages.append((message["id"],mime_msg))
    
            except: 
                print("error")
                pass
    return messages

In [8]:
def receive_all_messages(service:Resource):
    """ Recieve all messages and print them

        Args:
        service (Resource): Authorized Gmail API service instance.

    """
    messages = []
    # request a list of all the messages 
    result = service.users().messages().list(userId='me').execute() 
  
    # iterate through all the messages 
    for msg in result.get('messages') : 
        # Get the message from its id   
        # Use try-except to avoid any Errors 
        message = service.users().messages().get(userId="me", id=msg["id"],
                                             format='raw').execute()
        
        
        
        try:
            print(message["id"])
            msg_str = base64.urlsafe_b64decode(message['raw'].encode("utf-8")).decode("utf-8")
            print(msg_str)
            mime_msg = email.message_from_string(msg_str)

            messages.append((message["id"],mime_msg))

        except: 
            print("error")
            pass
    return messages

In [9]:
def print_message(message):
    """ Print the Content of a Message

    Args:
        message (email): The email encoded message
    """
    try:
        if message.is_multipart():
            for payload in message.get_payload():
                print(payload.get_payload())
        else:
            print(message.get_payload())
    except:
        message.get_payload()

In [10]:
def get_text_from_message(message):
    """ Print the Content of a Message

    Args:
        message (email): The email encoded message
    """
    retText=""
    try:
        if message.is_multipart():

            for payload in message.get_payload():
                retText+=str(payload.get_payload())
                retText+="\n"
        else:
            retText+=str(message.get_payload())
    except:
        message.get_payload()
    return retText

### Eigene Funktionen

In [11]:
def markAsRead(service:Resource, message_id):
    if type(message_id) is list:
        for mid in message_id:
            service.users().messages().modify(userId='me', id=mid, body={'removeLabelIds': ['UNREAD']}).execute()
    else:
        service.users().messages().modify(userId='me', id=message_id, body={'removeLabelIds': ['UNREAD']}).execute()
    
def deleteMail(service:Resource, message_id):
    if type(message_id) is list:
        for mid in message_id:
            service.users().messages().delete(userId="me", id=mid).execute()
    else:
        service.users().messages().delete(userId="me", id=message_id).execute()

In [12]:
# Nötig, wenn Nachrichten von Google reinkommen.
# deleteMail(service,"1777346d2ae8e766")

### Usage of the function

In [13]:
token_path = "token.pickle"

In [14]:
service = load_service(token_path)

In [15]:
messages = receive_all_messages(service)

177742649526489d
Delivered-To: team4@lumos-consulting.at
Received: by 2002:a05:7000:13d3:0:0:0:0 with SMTP id v19csp2254869mal;
        Fri, 5 Feb 2021 13:43:22 -0800 (PST)
X-Received: by 2002:a05:6a00:854:b029:1b7:6233:c5f with SMTP id q20-20020a056a000854b02901b762330c5fmr6564513pfk.73.1612561402318;
        Fri, 05 Feb 2021 13:43:22 -0800 (PST)
ARC-Seal: i=1; a=rsa-sha256; t=1612561402; cv=none;
        d=google.com; s=arc-20160816;
        b=YXczsLDktWh1shnoAO2MMokTCOkAqQLslL31yShhyLxbQFMmE3QX5hMrA9FxduQg7t
         93/34M+QblIzD+jP5ZEvWUAlbcLDvU9UsM8hI9Z6hSsF9RckLddtvto6eU7YEspsMAgQ
         3z90maJqin8N3rhnYl0kACprFk4fcUZbeHuE6MHoG90N6p8DxtsfydhiZ/Eq8yFmooWR
         EIb9MOpnW6cNXlbWpf3eDtd+EzwbO62p5nlV+qWXSv9AAG/jTnPG6a7g+W+mkiWF3Z9z
         ytcUMpFOJDDaNrQpdSguclmWgxFGczXpZiBJ3lARjm3hsc3bxbIzgJsRX7Hj7VFT4Baz
         9HAQ==
ARC-Message-Signature: i=1; a=rsa-sha256; c=relaxed/relaxed; d=google.com; s=arc-20160816;
        h=to:subject:message-id:date:from:mime-version:dkim-sign

1777421885035c8f
Received: from 918173104422
	named unknown
	by gmailapi.google.com
	with HTTPREST;
	Fri, 5 Feb 2021 13:38:10 -0800
Content-Type: text/html; charset="utf-8"
MIME-Version: 1.0
Content-Transfer-Encoding: base64
to: team4@lumos-consulting.at
from: team4@lumos-consulting.at
subject: Test1
Date: Fri, 5 Feb 2021 13:38:10 -0800
Message-Id: <CAEzfA9pkW0whinR3V5gA8dXfM6KLi_MDKt0OiO4XJ-6w+HP=Dw@mail.gmail.com>

VGFzZXJzdGVyw7Y=

177741c8cfd1913f
Received: from 918173104422
	named unknown
	by gmailapi.google.com
	with HTTPREST;
	Fri, 5 Feb 2021 13:32:44 -0800
Content-Type: text/html; charset="utf-8"
MIME-Version: 1.0
Content-Transfer-Encoding: base64
to: thomas.n71@gmail.com
from: team4@lumos-consulting.at
subject: Unklarheit bei Zuweisung: asdf123
Date: Fri, 5 Feb 2021 13:32:44 -0800
Message-Id: <CAEzfA9qe2OqQ8BXm0gBEaahTu+-LQ_KTsp0OAzZmOrHuVDn5Yw@mail.gmail.com>

bGVlZWVlZWVlZWVlZWVlZWVlZWVlZWVlZWVyb29vb29vb3kNCgo8ZGl2IGRpcj0ibHRyIj5sZWVl
ZWVlZWVlZWVlZWVlZWVlZWVlZWVlZXJvb29vb29v

17774110a77ca0c3
Received: from 918173104422
	named unknown
	by gmailapi.google.com
	with HTTPREST;
	Fri, 5 Feb 2021 13:20:09 -0800
Content-Type: text/html; charset="utf-8"
MIME-Version: 1.0
Content-Transfer-Encoding: base64
to: team4@lumos-consulting.at
from: team4@lumos-consulting.at
subject: Unklarheit bei Zuweisung: Unklarheit bei Zuweisung: Test_0
Date: Fri, 5 Feb 2021 13:20:09 -0800
Message-Id: <CAEzfA9qx5jKFEgGLv1n0rMevH5z-chUqMeGLaM=o9OmCeRB2+Q@mail.gmail.com>

VWtkR2VrbEhiSHBrUTBKcldsaEpaMXBZU25wa1IxVm5Wa2RXZW1STlQyc0tDZ3BmWDE5ZlgxOWZY
MTlmWDE5ZlgxOWZYMTlmWDE5ZgpYMTlmWDE5ZlgxOWZYMTlmWDE5ZlgxOWZYMThLUkdsbGMyVWdU
V0ZwYkNCcGMzUWdiV2wwSURVMUxqa3pJQ1VnVjJGb2NuTmphR1ZwCmJteHBZMmhyWldsMElHYkR2
SElnUVdKMFpXbHNkVzVuSURZZ1ltVnpkR2x0YlhRdUNncE5hWFFnTXpBdU16UWdKU0JYWVdoeWMy
Tm8KWldsdWJHbGphR3RsYVhRZ2E4TzJibTUwWlNCVGFXVWdac084Y2lCQlluUmxhV3gxYm1jZ05T
QjFibVFnYldsMElEWXVNemdnSlNCWApZV2h5YzJOb1pXbHViR2xqYUd0bGFYUWdac084Y2lCQllu
UmxhV3gxYm1jZ01pQmlaWE4wYVcxdGRDQnpaV2x1TGc9PQoKX19fX19fX19fX19f

17774109bf6de03f
Received: from 918173104422
	named unknown
	by gmailapi.google.com
	with HTTPREST;
	Fri, 5 Feb 2021 13:19:41 -0800
Content-Type: text/html; charset="utf-8"
MIME-Version: 1.0
Content-Transfer-Encoding: base64
to: team4@lumos-consulting.at
from: team4@lumos-consulting.at
subject: Unklarheit bei Zuweisung: Unklarheit bei Zuweisung: Test_0
Date: Fri, 5 Feb 2021 13:19:41 -0800
Message-Id: <CAEzfA9qSDuoWM7URmT9wQYspLh31bG=CY1uYYw5-m6kVOeyVag@mail.gmail.com>

VWtkR2VrbEhiSHBrUTBKcldsaEpaMXBZU25wa1IxVm5Wa2RXZW1SQlBUMEtDbDlmWDE5ZlgxOWZY
MTlmWDE5ZlgxOWZYMTlmWDE5ZgpYMTlmWDE5ZlgxOWZYMTlmWDE5ZlgxOWZYd3BFYVdWelpTQk5Z
V2xzSUdsemRDQnRhWFFnTmpRdU9Ea2dKU0JYWVdoeWMyTm9aV2x1CmJHbGphR3RsYVhRZ1pzTzhj
aUJCWW5SbGFXeDFibWNnTmlCaVpYTjBhVzF0ZEM0S0NrMXBkQ0F4T1M0eE5DQWxJRmRoYUhKelky
aGwKYVc1c2FXTm9hMlZwZENCcnc3WnViblJsSUZOcFpTQm13N3h5SUVGaWRHVnBiSFZ1WnlBMUlI
VnVaQ0J0YVhRZ05pNHhOU0FsSUZkaAphSEp6WTJobGFXNXNhV05vYTJWcGRDQm13N3h5SUVGaWRH
VnBiSFZ1WnlBeUlHSmxjM1JwYlcxMElITmxhVzR1CgpfX19fX19fX19fX19fX19f

177740c4d0a68e1b
Received: from 918173104422
	named unknown
	by gmailapi.google.com
	with HTTPREST;
	Fri, 5 Feb 2021 13:14:59 -0800
Content-Type: text/html; charset="utf-8"
MIME-Version: 1.0
Content-Transfer-Encoding: base64
to: team4@lumos-consulting.at
from: team4@lumos-consulting.at
subject: Unklarheit bei Zuweisung: Test_1
Date: Fri, 5 Feb 2021 13:14:59 -0800
Message-Id: <CAEzfA9rnPbn3_uod53ZTb40c8FJ2VY1wg-J-sO+sCqGn2KirwQ@mail.gmail.com>

U1dOb0lHaGhZbVVnVjBGU1JVaFBWVk5GSUZCeWIyUjFhM1JsSUdsdElFbFVKazlHUmtsRFJVdGhk
R0ZzYjJjeE5TQm1hWEp6ZEdOcwpZWE56SUhOMFlXNWtZWEprYldGbGMzTnBaeUJwYm5OMFlXeHNh
V1Z5ZEM0Z1JHbGxJRWx1YzNSaGJHeGhkR2x2YmlCc2FXVm1JR1psCmFHeGxjbVp5WldrZ1lXSWdk
VzVrSUhkMWNtUmxJRzl5Wkc1MWJtZHpaMlZ0WVdWemN5QmhZbWRsYzJOb2JHOXpjMlZ1TGlCRVlY
TWcKVUhKdlozSmhiVzBnZDNWeVpHVWdZVzV6WTJoc2FXVnpjMlZ1WkNCeVpXZHBjM1J5YVdWeWRD
NGdJRUpsYVcwZ1VISnZaM0poYlcxaApkV1p5ZFdZZ2EyOXRiWFFnWm5WbGNpQmpZUzRnTkMwMUlG
TmxhM1Z1WkdWdUlHUnBaU0JUWVc1a2RXaHlJR0YxWmlCa1pXMGdRbWxzClpITmphR2x5YlN3Z2My
OXVjM1FnYTJWc

177740bb747225a7
Received: from 918173104422
	named unknown
	by gmailapi.google.com
	with HTTPREST;
	Fri, 5 Feb 2021 13:14:20 -0800
Content-Type: text/html; charset="utf-8"
MIME-Version: 1.0
Content-Transfer-Encoding: base64
to: team4@lumos-consulting.at
from: team4@lumos-consulting.at
subject: Unklarheit bei Zuweisung: Test_0
Date: Fri, 5 Feb 2021 13:14:20 -0800
Message-Id: <CAEzfA9qG-ngPKb3Dj+2MyzQwzM4hUcjPtY2=iGL+T9FbWD1gyA@mail.gmail.com>

UkdGeklHbHpkQ0JrWlhJZ1pYSnpkR1VnVkdWemRNT2sKCl9fX19fX19fX19fX19fX19fX19fX19f
X19fX19fX19fX19fX19fX19fXwpEaWVzZSBNYWlsIGlzdCBtaXQgNTUuOTMgJSBXYWhyc2NoZWlu
bGljaGtlaXQgZsO8ciBBYnRlaWx1bmcgNiBiZXN0aW1tdC4KCk1pdCAzMC4zNCAlIFdhaHJzY2hl
aW5saWNoa2VpdCBrw7ZubnRlIFNpZSBmw7xyIEFidGVpbHVuZyA1IHVuZCBtaXQgNi4zOCAlIFdh
aHJzY2hlaW5saWNoa2VpdCBmw7xyIEFidGVpbHVuZyAyIGJlc3RpbW10IHNlaW4u

177740bab26ab8ca
Received: from 918173104422
	named unknown
	by gmailapi.google.com
	with HTTPREST;
	Fri, 5 Feb 2021 13:14:17 -0800
Content-Type: text/html; charset="utf-8"
MIME-

In [16]:
#message = create_message("team4@lumos-consulting.at", "team4@lumos-consulting.at", "Test1","Tasersterö")


Content-Type: multipart/mixed; boundary="===============6414113519386042882=="
MIME-Version: 1.0
to: team4@lumos-consulting.at
from: team4@lumos-consulting.at
subject: Test1

--===============6414113519386042882==
Content-Type: text/plain; charset="utf-8"
MIME-Version: 1.0
Content-Transfer-Encoding: base64

VGFzZXJzdGVyw7Y=

--===============6414113519386042882==--

b'Q29udGVudC1UeXBlOiBtdWx0aXBhcnQvbWl4ZWQ7IGJvdW5kYXJ5PSI9PT09PT09PT09PT09PT02NDE0MTEzNTE5Mzg2MDQyODgyPT0iCk1JTUUtVmVyc2lvbjogMS4wCnRvOiB0ZWFtNEBsdW1vcy1jb25zdWx0aW5nLmF0CmZyb206IHRlYW00QGx1bW9zLWNvbnN1bHRpbmcuYXQKc3ViamVjdDogVGVzdDEKCi0tPT09PT09PT09PT09PT09NjQxNDExMzUxOTM4NjA0Mjg4Mj09CkNvbnRlbnQtVHlwZTogdGV4dC9wbGFpbjsgY2hhcnNldD0idXRmLTgiCk1JTUUtVmVyc2lvbjogMS4wCkNvbnRlbnQtVHJhbnNmZXItRW5jb2Rpbmc6IGJhc2U2NAoKVkdGelpYSnpkR1Z5dzdZPQoKLS09PT09PT09PT09PT09PT02NDE0MTEzNTE5Mzg2MDQyODgyPT0tLQo='
Q29udGVudC1UeXBlOiBtdWx0aXBhcnQvbWl4ZWQ7IGJvdW5kYXJ5PSI9PT09PT09PT09PT09PT02NDE0MTEzNTE5Mzg2MDQyODgyPT0iCk1JTUUtVmVyc2lvbjogMS4wCnRvOiB

In [17]:
#send_message(service, "me", message)

Message Id: 177742674f1c66a1


In [18]:
messages = receive_unread_messages(service)


177742674f1c66a1
Received: from 918173104422
	named unknown
	by gmailapi.google.com
	with HTTPREST;
	Fri, 5 Feb 2021 13:43:33 -0800
Content-Type: multipart/mixed; boundary="===============6414113519386042882=="
MIME-Version: 1.0
to: team4@lumos-consulting.at
from: team4@lumos-consulting.at
subject: Test1
Date: Fri, 5 Feb 2021 13:43:33 -0800
Message-Id: <CAEzfA9r08XPM07m-vj3oxPb9HTGTyjy0Uttpu+977zZFRfRv1Q@mail.gmail.com>

--===============6414113519386042882==
Content-Type: text/plain; charset="utf-8"
MIME-Version: 1.0
Content-Transfer-Encoding: base64

VGFzZXJzdGVyw7Y=

--===============6414113519386042882==--

177742649526489d
Delivered-To: team4@lumos-consulting.at
Received: by 2002:a05:7000:13d3:0:0:0:0 with SMTP id v19csp2254869mal;
        Fri, 5 Feb 2021 13:43:22 -0800 (PST)
X-Received: by 2002:a05:6a00:854:b029:1b7:6233:c5f with SMTP id q20-20020a056a000854b02901b762330c5fmr6564513pfk.73.1612561402318;
        Fri, 05 Feb 2021 13:43:22 -0800 (PST)
ARC-Seal: i=1; a=rsa-sha256; t=

In [19]:
labellist=np.arange(1,7)
PATH="C:\\Users\\Tom\\Google Drive\\data\\final_model"
from transformers import AutoModelForSequenceClassification, AutoTokenizer

def predictAbteilung(text):
    model = AutoModelForSequenceClassification.from_pretrained(PATH)
    tokenizer = AutoTokenizer.from_pretrained(PATH)
    labels = torch.tensor([1]).unsqueeze(0)
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    prediction = model(**inputs, labels=labels)
    logits=prediction.logits.detach().numpy()[0]
    probabilities=softmax(logits)
    print(probabilities)
    return arg3max(probabilities)

In [20]:
def arg3max(liste):
    retList = [(0,0),(0,0),(0,0)]
    print(liste)
    for (count,value) in enumerate(liste):
        if value > retList[0][1]:
            if value > retList[1][1]:
                if value > retList[2][1]:
                    retList[0]=retList[1]
                    retList[1]=retList[2]
                    retList[2]=(count,value)
                else:
                    retList[0]=retList[1]
                    retList[1]=(count,value)
            else:
                retList[0]=(count,value)
    return retList
        
def mail_weiterleitung(service):
    
    messages = receive_unread_messages(service)
    
    for mid,message in messages:
        text=get_text_from_message(message)
        print(100*"*")
        print(text)
        predictions = predictAbteilung(text)
        abteilung1 = labellist[predictions[2][0]]
        abteilung2 = labellist[predictions[1][0]]
        abteilung3 = labellist[predictions[0][0]]
        text+="\n__________________________________________\\nDiese Mail ist mit {p1:.2f} % Wahrscheinlichkeit für Abteilung {a1} bestimmt.\n\nMit {p2:.2f} % Wahrscheinlichkeit könnte Sie für Abteilung {a2} und mit {p3:.2f} % Wahrscheinlichkeit für Abteilung {a3} bestimmt sein.".format(p1=predictions[2][1]*100,p2=predictions[1][1]*100,p3=predictions[0][1]*100,a1=str(abteilung1),a2=str(abteilung2),a3=str(abteilung3))
        if predictions[2][1] > 0.9:
            betreff="Abteilung "+str(abteilung1)+":  "+str(message["subject"])
        else:
            betreff="Unklarheit bei Zuweisung: "+str(message["subject"])
        wl_message = create_message("team4@lumos-consulting.at", "thomas.n71@gmail.com", betreff, text)
        send_message(service, "me", wl_message)
        markAsRead(service,mid)

In [21]:
import time
while True:
    
    mail_weiterleitung(service)
    time.sleep(5)

177742674f1c66a1
Received: from 918173104422
	named unknown
	by gmailapi.google.com
	with HTTPREST;
	Fri, 5 Feb 2021 13:43:33 -0800
Content-Type: multipart/mixed; boundary="===============6414113519386042882=="
MIME-Version: 1.0
to: team4@lumos-consulting.at
from: team4@lumos-consulting.at
subject: Test1
Date: Fri, 5 Feb 2021 13:43:33 -0800
Message-Id: <CAEzfA9r08XPM07m-vj3oxPb9HTGTyjy0Uttpu+977zZFRfRv1Q@mail.gmail.com>

--===============6414113519386042882==
Content-Type: text/plain; charset="utf-8"
MIME-Version: 1.0
Content-Transfer-Encoding: base64

VGFzZXJzdGVyw7Y=

--===============6414113519386042882==--

177742649526489d
Delivered-To: team4@lumos-consulting.at
Received: by 2002:a05:7000:13d3:0:0:0:0 with SMTP id v19csp2254869mal;
        Fri, 5 Feb 2021 13:43:22 -0800 (PST)
X-Received: by 2002:a05:6a00:854:b029:1b7:6233:c5f with SMTP id q20-20020a056a000854b02901b762330c5fmr6564513pfk.73.1612561402318;
        Fri, 05 Feb 2021 13:43:22 -0800 (PST)
ARC-Seal: i=1; a=rsa-sha256; t=

Message Id: 1777426bc4b9ee69
****************************************************************************************************
d=C3=A4=C3=A4=C3=A4=C3=A4=C3=A4=C3=A4h

--=20
Leystra=C3=9Fe 163
A-1020 Wien
Mobil: 0043-664-9172427
E-Mail: thomas.lukas.niedermayer@gmail.com

<div dir=3D"ltr">d=C3=A4=C3=A4=C3=A4=C3=A4=C3=A4=C3=A4h<br clear=3D"all"><d=
iv><br></div>-- <br><div dir=3D"ltr" class=3D"gmail_signature" data-smartma=
il=3D"gmail_signature"><div dir=3D"ltr"><div>Leystra=C3=9Fe 163<br>A-1020 W=
ien<br>Mobil: 0043-664-9172427<br>E-Mail: <a href=3D"mailto:thomas.lukas.ni=
edermayer@gmail.com" target=3D"_blank">thomas.lukas.niedermayer@gmail.com</=
a></div></div></div></div>


[1.3176524e-03 8.2545955e-04 6.9426396e-04 5.4375920e-04 1.8635782e-03
 9.9475527e-01]
[1.3176524e-03 8.2545955e-04 6.9426396e-04 5.4375920e-04 1.8635782e-03
 9.9475527e-01]
Content-Type: multipart/mixed; boundary="===============4828828133631995184=="
MIME-Version: 1.0
to: thomas.n71@gmail.com
from: team4@lum

KeyboardInterrupt: 